In [ ]:
%maven com.google.guava:guava:29.0-jre
%maven net.jqwik:jqwik:1.3.5
%maven org.hamcrest:hamcrest:2.2
%maven org.junit.jupiter:junit-jupiter-api:5.6.2
%maven org.junit.jupiter:junit-jupiter-params:5.6.2
%maven org.junit.jupiter:junit-jupiter-engine:5.6.2
%maven org.junit.platform:junit-platform-launcher:1.6.2

In [ ]:
import static org.junit.jupiter.api.Assertions.assertTrue;
import org.junit.jupiter.params.ParameterizedTest;
import org.junit.jupiter.params.provider.Arguments;
import org.junit.jupiter.params.provider.MethodSource;
import org.junit.jupiter.params.provider.ValueSource;
import static org.junit.jupiter.api.Assertions.assertEquals;
import static org.junit.jupiter.api.Assertions.assertThrows;
import static org.junit.jupiter.api.Assertions.assertTrue;
import org.junit.jupiter.api.Test;
import java.util.stream.Stream;
import net.jqwik.api.*;
import net.jqwik.api.constraints.*;
import static org.hamcrest.MatcherAssert.assertThat;
import static org.hamcrest.Matchers.is;
import static org.hamcrest.Matchers.anyOf;

In [ ]:
public class Min {

    public static int min(int a, int b) {
        if (a < b)
            return a;
        else
            return b;
    }
}

In [ ]:
public class TestMin {

    @Test
    public void testmin1() {
        assertEquals(0, Min.min(0,1));
    }

    @Test
    public void testshouldfail() {
        assertEquals(34, Min.min(0,1));
    }

    @Test
    public void failAssertion() {
        assert false;
    }
}


In [ ]:
// Adapted from https://www.baeldung.com/junit-tests-run-programmatically-from-java
// https://github.com/eugenp/tutorials/blob/master/testing-modules/junit-5/src/test/java/com/baeldung/runfromjava/RunJUnit5TestsFromJava.java

import org.junit.platform.launcher.Launcher;
import org.junit.platform.launcher.LauncherDiscoveryRequest;
import org.junit.platform.launcher.TestPlan;
import org.junit.platform.launcher.core.LauncherDiscoveryRequestBuilder;
import org.junit.platform.launcher.core.LauncherFactory;
import org.junit.platform.launcher.listeners.SummaryGeneratingListener;
import org.junit.platform.launcher.listeners.TestExecutionSummary;

import java.io.PrintWriter;

import static org.junit.platform.engine.discovery.ClassNameFilter.includeClassNamePatterns;
import static org.junit.platform.engine.discovery.DiscoverySelectors.selectClass;
import static org.junit.platform.engine.discovery.DiscoverySelectors.selectPackage;

public class RunJUnit5TestsFromJava {
    SummaryGeneratingListener listener = new SummaryGeneratingListener();
     
    public void runOne(Class clz) {
        LauncherDiscoveryRequest request = LauncherDiscoveryRequestBuilder.request()
          .selectors(selectClass(clz))
          .build();
        Launcher launcher = LauncherFactory.create();
        TestPlan testPlan = launcher.discover(request);
        launcher.registerTestExecutionListeners(listener);
        launcher.execute(request);
    }
    
    public void run(Class clz) {
        run(clz, false);
    }

    public void run(Class clz, boolean verbose) {
        RunJUnit5TestsFromJava runner = new RunJUnit5TestsFromJava();
        runner.runOne(clz);
 
        TestExecutionSummary summary = runner.listener.getSummary();
        summary.printTo(new PrintWriter(System.out));
        for (var failure : summary.getFailures()) {
            System.out.println(failure.getTestIdentifier().getDisplayName());
            if (verbose) {
                failure.getException().printStackTrace(System.out);
            } else {
                System.out.println(failure.getException().getMessage());
            }
        }
    }
}

In [ ]:
RunJUnit5TestsFromJava runner = new RunJUnit5TestsFromJava();
runner.run(TestMin.class);

In [ ]:
public class MinMatchers {

    @Test
    public void matchStuff() {
        assertThat(Min.min(0,4), is(0));

        assertThat(Min.min(3,4), anyOf(is(9), is(4)));
    }

}

In [ ]:
runner.run(MinMatchers.class);

In [ ]:
public class Exceptions {

    @Test
    public void testExceptions() {
    Exception exception = assertThrows(UnsupportedOperationException.class,
            /* The bit between this and the next comment is a 
             * *lambda expression*: https://docs.oracle.com/javase/tutorial/java/javaOO/lambdaexpressions.html#syntax
             * If you've taken PL, this is Java's lambda.
             * If not, this is like special syntax for a function object with one method.
             * Notice the form is:
             *   _args_ -> { body } */
            () -> { throw new UnsupportedOperationException("Bad jasdf;lkasdjlfk"); }
            /* end of lambda expression. The () at the beginning is the (empty)
             * argument list of the lambda */
            ); // <-- This is closing off the call to assertThrows
    // Here you can use any methods on the Exception type to inspect whether the right exception was thrown.
        assertEquals("Bad Operation", exception.getMessage());   
    }

}


In [ ]:
runner.run(Exceptions.class);

In [ ]:
public class TestMinTheory {

    @ParameterizedTest
    @ValueSource(ints = {Integer.MIN_VALUE, -4, 0, 6, Integer.MAX_VALUE})
    public void testMinTheories(int a, int b){
        int c = Min.min(a,b);
        assertTrue(c <= a, "min less than first argument");
        assertTrue(c <= b, "min less than second argument");
        assertTrue(c == a || c == b, "min is one of the arguments");
    }


    @ParameterizedTest
    @MethodSource("provideSomeOddTests")
    void testOdd(int input, boolean expected) {
        assertEquals(expected, input % 2 == 1);
    }

    private static Stream<Arguments> provideSomeOddTests() {
        return Stream.of(
          Arguments.of(2, false),
          Arguments.of(4, false),
          Arguments.of(3, true),
          Arguments.of(5, true)
        );
    }

}

In [ ]:
runner.run(TestMinTheory.class);

In [ ]:
public class PropertyTests {

    @Property(tries = 250, shrinking = ShrinkingMode.BOUNDED)
    public boolean checkMinRight(@ForAll int x, @ForAll int y) {
        Assume.that(x > y);
        return Min.min(x,y) == y;
    }
    @Property(tries = 250, shrinking = ShrinkingMode.BOUNDED)
    public boolean checkMinLeft(@ForAll int x, @ForAll int y) {
        Assume.that(x < y);
        return Min.min(x,y) == y;
    }
}

In [ ]:
runner.run(PropertyTests.class);